<a href="https://colab.research.google.com/github/0x1beef/uap/blob/main/nb/interlacing.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a href="https://kaggle.com/kernels/welcome?src=https://github.com/0x1beef/uap/blob/main/nb/interlacing.ipynb">
    <img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"   />
</a>


In [ ]:
import cv2
import numpy as np
import os

(w, h) = (480, 480)
step = 8
num_frames = 5 * step

back_colors = [60, 120]
def get_back_color(frame):
    back_color = back_colors[0]
    if frame == step or frame == step + 1:
        back_color = back_colors[1]
    if frame >= 2 * step:
        back_color = back_colors[1]
    if frame == 3 * step or frame == 3 * step + 1:
        back_color = back_colors[0]
    if frame >= 4 * step:
        back_color = back_colors[0]
    return back_color

bar_offsets = [100, 120]
def get_bar_offset(frame):
    bar_ofs = bar_offsets[0]
    if frame == step/2 or frame == step/2 + 1:
        bar_ofs = bar_offsets[1]
    if frame >= 2 * step - step / 2:
        bar_ofs = bar_offsets[1]
    if frame == 3 * step - step/2 or frame == 3 * step - step/2 + 1:
        bar_ofs = bar_offsets[0]
    if frame >= 4 * step - step / 2:
        bar_ofs = bar_offsets[0]
    return bar_ofs

def get_ellipse_rot_rect(frame):
    el_frame = int(frame/2)*2
    #el_frame = frame
    angle = (45 - el_frame * 3 / 2)
    period = 14; move_per_frame = 1
    loop_frame = period - abs(period - (el_frame % (2 * period)))
    x = w / 2 + loop_frame * move_per_frame - (period * move_per_frame / 2)
    y = h / 2 + (abs(period / 2 - loop_frame) - period / 2)
    return cv2.RotatedRect((x,y), (8, 25), angle) 

os.makedirs("test/in", exist_ok=True )
for frame in range(0, num_frames):
    mat = np.full((h,w), get_back_color(frame), np.uint8)
    bar_ofs = get_bar_offset(frame)
    for i in range(int(h / 4), int(3 * h / 4)):
        for j in range(0, 5):
            mat[i, int(w / 2) - bar_ofs + j] = 255
            mat[i, int(w / 2) + bar_ofs + j] = 255
    cv2.ellipse(mat, get_ellipse_rot_rect(frame), 0, -1)
    cv2.imwrite(f"test/in/test_{frame:04d}.png", mat)


\begin{align}
I_3(i,j,t) & =
  \begin{cases}
  I_2(i,j, 2 t+1) & \text {if $i$ is even, } \\
  I_2(i,j, 2 t) & \text { if $i$ is odd }
  \end{cases} \\
s(x) & = (x+0.5) \dfrac{480}{428} \\
I_2(i,j,t) & \approx \sum\limits_{(i_n,j_n) \in N(s(i),s(j))} c_h(j,j_n)c_v(i,i_n) I_1(i_n,j_n,t) \\
I_1(i,j,t) & =
  \begin{cases}
  I(i,j,t) & \text{if ($t$ is even and $i$ is even) or ($t$ is odd and $i$ is odd), } \\
  I(i,j,t+1) & \text{otherwise}
  \end{cases}
\end{align}


In [ ]:
import os;
ENCODE="-c:v libx264 -preset veryslow -qp 0 -crf 0" # fully lossless video encoding
ENCODE_WMV="-codec wmv1 -b:v 3M" # try to match the original quality
VID_FULL="test_vid_full.wmv"

INPUT_PNG="-start_number 0 -i test/in/test_%04d.png"
# create a 2 FPS video
os.system(f'ffmpeg -r 2 {INPUT_PNG} -vf "format=gray" {ENCODE} -r 2 -y test_vid.mp4')

# using tinterlace filters (https://ffmpeg.org/ffmpeg-filters.html#tinterlace)
# that implement going from I to I1 and going from I2 to I3
INTERLACE_BOTTOM="tinterlace=interleave_bottom, tinterlace=interlacex2"
INTERLACE_TOP="tinterlace=interleave_top, tinterlace=interlacex2"
SCALE="scale=428:428:sws_flags=bilinear"
SELECT_EVEN="select='eq(mod(n,2),0)'" # drop every odd output frame
FULL_FILTERS=f"{INTERLACE_TOP}, {SCALE}, {INTERLACE_BOTTOM}, {SELECT_EVEN}"
# create a 1 FPS video from a 2 FPS input
os.system(f'ffmpeg -r 2 {INPUT_PNG} -vf "{FULL_FILTERS}" {ENCODE_WMV} -r 1 -y {VID_FULL}')
os.system(f'ffmpeg -i {VID_FULL} {ENCODE} -y test_vid_full.mp4')

SCALE_VERT="scale=iw:2*ih:sws_flags=neighbor"
FIELD_TOP="setfield=tff, field=0" # make sure the field dominance is set to 'tff' to get consistent results
FIELD_BOTTOM="setfield=tff, field=1"
os.system(f'ffmpeg -i {VID_FULL} -vf "{FIELD_TOP}, {SCALE_VERT}" {ENCODE} -y test_vid_full_top.mp4')
os.system(f'ffmpeg -i {VID_FULL} -vf "{FIELD_BOTTOM}, {SCALE_VERT}" {ENCODE} -y test_vid_full_bottom.mp4')

os.makedirs("test/out", exist_ok=True )
os.system(f'ffmpeg -i {VID_FULL} -start_number 0 test/out/frame_%04d.png')


In [ ]:
import os
from huggingface_hub import hf_hub_download

gimbal_vid = "gimbal/2 - GIMBAL.wmv"
hf_hub_download(repo_id="logicbear/uap", filename=gimbal_vid, repo_type="dataset", local_dir="./")

format_crop = "format=gray, crop=428:428:104:27"
os.system(f'ffmpeg -i "{gimbal_vid}" -vf "{format_crop}" -start_number 0 gimbal/frame_%04d.png')


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

#gimbal_frame=980
#gimbal_frame=373
#gimbal_frame=981
gimbal_frame=1022
#gimbal_frame=1023

test_frame = 11 if gimbal_frame % 2 == 0 else 12

mat = cv2.imread(f"gimbal/frame_{gimbal_frame:04d}.png", cv2.IMREAD_GRAYSCALE)
mat_test = cv2.imread(f"test/out/frame_{test_frame:04d}.png", cv2.IMREAD_GRAYSCALE)
(h, w) = mat.shape
rows = range(0,h,2)
cols = slice(0,w)
#cols = slice(int(w/2)+20,w)
#cols = slice(0,int(w/2)-20)

gimbal_odd = [np.mean(mat[i+1, cols]) for i in rows]
gimbal_even = [np.mean(mat[i, cols]) for i in rows]
gimbal_diff = [np.mean(mat[i, cols]) - np.mean(mat[i+1, cols]) for i in rows]

scale=4
shift=3 if gimbal_frame % 2 == 0 else -3
#todo: the test diff is scaled and shifted to match, so this only tests frequency not amplitude
test_diff = [(np.mean(mat_test[i, cols]) - np.mean(mat_test[i+1, cols]))/scale+shift for i in rows]

plt.figure(figsize=(20,10))
plt.title(f"gimbal frame {gimbal_frame} vs test frame {test_frame}", fontsize=20)
plt.plot(gimbal_odd, color='royalblue', label='gimbal odd rows')
plt.plot(gimbal_even, color='darkred', label='gimbal even rows')
plt.plot(gimbal_diff, color='crimson', label='gimbal even - odd')
plt.plot(test_diff, color='indigo', label='test even - odd')
plt.xlabel("row / 2", fontsize=20)
plt.ylabel("mean pixel intensity", fontsize=20)
plt.legend(fontsize=20)
#plt.savefig('plot_bands.png')
plt.show()


In [ ]:
!pip install mediapy


In [ ]:
import cv2
import numpy as np
import mediapy as media
from dataclasses import dataclass

@dataclass
class Roi:
    top:int; bottom: int; left:int; right: int
                    
def image_label(img, label, pos):
    (rows,cols) = img.shape; (y,x) = pos
    font = cv2.FONT_HERSHEY_SIMPLEX; scale = 1; color = 255
    (text_w, text_h), baseLine = cv2.getTextSize(label, font, scale, 1)
    y = rows + y if y < 0 else y + text_h
    x = cols - text_w + x if x < 0 else x
    cv2.putText(img, label, (x, y), font, scale, color)

def get_fields(mat, roi : Roi, resize_factor, frame, label_pos = (-2, 1)):
    mat_top = mat[roi.top:roi.bottom:2, roi.left:roi.right]
    mat_bottom = mat[roi.top+1:roi.bottom+1:2, roi.left:roi.right]
    # duplicate every other row to keep the aspect ratio
    def resize(mat, fxy) :
        return cv2.resize(mat, (0,0), fx = fxy, fy = 2 * fxy, interpolation = cv2.INTER_NEAREST)
    mat_top = resize(mat_top, resize_factor)
    mat_bottom = resize(mat_bottom, resize_factor)
    image_label(mat_top, f'{frame} T', label_pos)
    image_label(mat_bottom, f'{frame} B', label_pos)
    return (mat_top, mat_bottom)

def append_fields(frame_list, mat_top, mat_bottom):
    #frame_list.append(mat_top)
    frame_list.append(mat_bottom)
    frame_list.append(mat_top)

frames_gimbal = []
for frame in range(1001,1007):
    mat = cv2.imread(f"gimbal/frame_{frame:04d}.png", cv2.IMREAD_GRAYSCALE)
    (rows,cols) = mat.shape
    roi_factor = 8
    # make sure these offsets start from an even number for top/bottom to be right
    roi_ofs = lambda dim, sign: int((dim/2 + sign * rows/roi_factor)/2)*2
    roi = Roi(top = roi_ofs(rows,-1), bottom = roi_ofs(rows,1),
        left = roi_ofs(cols,-1), right = roi_ofs(cols,1))
    (mat_top, mat_bottom) = get_fields(mat, roi, 2, frame)
    append_fields(frames_gimbal, mat_top, mat_bottom)
    
media.write_video("gimbal_field_order.gif", np.array(frames_gimbal), fps=2, codec='gif')
media.show_video(media.read_video("gimbal_field_order.gif"))

def get_full_roi(mat):
    (rows,cols) = mat.shape
    return Roi(top = 0, bottom = rows, left = 0, right = cols)

def append_full_fields(frame_list, mat, resize_factor = 1, label_pos = (-2, 1)):
    (mat_top, mat_bottom) = get_fields(mat, get_full_roi(mat), resize_factor, frame, label_pos)
    append_fields(frame_list, mat_top, mat_bottom)
    
frames_switch = []
for frame in range(372, 375):
    mat = cv2.imread(f"gimbal/frame_{frame:04d}.png", cv2.IMREAD_GRAYSCALE)
    append_full_fields(frames_switch, mat, label_pos=(1,-1))
    
media.write_video("switch_field_order.mp4", np.array(frames_switch), fps=1, codec='h264', crf=20)
media.show_video(media.read_video("switch_field_order.mp4"))
    
frames_test = []
for frame in range(0, 18):
    mat = cv2.imread(f"test/out/frame_{frame:04d}.png", cv2.IMREAD_GRAYSCALE)
    append_full_fields(frames_test, mat, label_pos=(1,1))

media.write_video("test_field_order.mp4", np.array(frames_test), fps=2, codec='h264', crf=0)
media.show_video(media.read_video("test_field_order.mp4"))

frames_desync = []
for frame in range(1021, 1024):
    mat = cv2.imread(f"gimbal/frame_{frame:04d}.png", cv2.IMREAD_GRAYSCALE)
    (mat_top, mat_bottom) = get_fields(mat, get_full_roi(mat), 1, frame, (-80,1))
    (rows,cols) = mat_bottom.shape
    mat_bottom = cv2.resize(mat_bottom[int(3*rows/4):rows], (0,0), fx=2, fy=2, interpolation=cv2.INTER_NEAREST)
    frames_desync.append(mat_bottom)
    
media.write_video("desync.mp4", np.array(frames_desync), fps=1, codec='h264', crf=0)
media.show_video(media.read_video("desync.mp4"))


In [ ]:
import os
from huggingface_hub import hf_hub_download

if not os.path.exists("flir1/frame_0000.png"):
    flir1_vid = "flir1/1 - FLIR.mp4"
    hf_hub_download(repo_id="logicbear/uap", filename=flir1_vid, repo_type="dataset", local_dir="./")
    flir1_filters = "format=gray, crop=240:238:54:15"
    os.system(f'ffmpeg -i "{flir1_vid}" -vf "{flir1_filters}" -start_number 0 flir1/frame_%04d.png')
    
frames_flir1 = []
for frame in range(1237, 1243):
    mat = cv2.imread(f"flir1/frame_{frame:04d}.png", cv2.IMREAD_GRAYSCALE)
    image_label(mat, f'{frame}', pos=(1,-1))
    frames_flir1.append(mat)
    
media.write_video("flir1_artifacts.mp4", np.array(frames_flir1), fps=1, codec='h264', crf=0)
media.show_video(media.read_video("flir1_artifacts.mp4"))
